#### Code written by Mohamed Abdallah

In [2]:
# Authenticate with Hugging Face
from huggingface_hub import login
login(token="hf_...")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Setting Directory for the uploaded files
! mkdir '/content/Docs'

### Installing required packages

In [ ]:
!pip install --upgrade sentence-transformers sentencepiece datasets

In [ ]:
!pip install openai langchain chromadb tiktoken langchain-community langchain_chroma langchain_openai ollama unstructured bitsandbytes

### Import Packages and Libraries

In [4]:
import os
import re
import pandas as pd
import numpy as np
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import openai
from langchain import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

### Reading table of Q&A csv file

In [5]:
df = pd.read_csv('/content/Retrieval_Data_mod_v4.csv')

In [6]:
df.columns

Index(['Retrieved_true', 'Questions', 'paraphrases',
       'eg_ar_question (GPT3.5 Turbo)', 'eg_ar_question_edited ',
       'answer (GPT3.5 Turbo)', 'Unnamed: 6', 'Unnamed: 7', 'Answers',
       'Generated_Answers_Mistral_7b', 'Eval_Answers_Mistral_7b',
       'generated_tokens', 'true_tokens', 'bleu_score_mistral',
       'bleu_score_mistral.1', 'Generated_Answers_llama_70b'],
      dtype='object')

---

### Uploading files, Splitting and Use various retrievals

In [7]:
loader = DirectoryLoader('/content/Docs')
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [ ]:
splits

### New Experiment

###  5. E5 (dim_size= 1024)

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Function to generate embeddings
def embed_texts(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().tolist())
    return embeddings


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
class E5Embeddings:
    def embed_documents(self, texts):
        return embed_texts(texts)

    def embed_query(self, text):
        return embed_texts([text])[0]

embedding_E5 = E5Embeddings()

In [11]:
# Create Chroma vectorstore with embeddings from the model
vectorstore_E5 = Chroma.from_documents(documents=splits, embedding=embedding_E5)
retriever_E5 = vectorstore_E5.as_retriever(search_type="similarity", search_kwargs={'k': 5})

### Answers Generation

In [51]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
from transformers import TextGenerationPipeline, BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import torch
from langchain.chains import RetrievalQA
from transformers import pipeline

In [13]:
import gc
gc.collect()

0

In [14]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,

)

In [49]:
jais_tokenizer = AutoTokenizer.from_pretrained("airev-ai/Jais-Inception-7b-V0.2",quantization_config=quantization_config,trust_remote_code=True)
jais_model = AutoModelForCausalLM.from_pretrained( "airev-ai/Jais-Inception-7b-V0.2",quantization_config=quantization_config,trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!mkdir /content/drive/MyDrive/JAIS_7b_q

mkdir: cannot create directory ‘/content/drive/MyDrive/JAIS_7b_q’: File exists


In [ ]:
# drive_path = '/content/drive/MyDrive/JAIS_7b_q'
# jais_tokenizer.save_pretrained(drive_path)
# jais_model.save_pretrained(drive_path)

In [52]:
pipeline = pipeline(
       "text-generation",
        model=jais_model,
        tokenizer=jais_tokenizer,
        truncation =True,
        device_map="auto",
        max_new_tokens=2000,
        do_sample=True,
        top_k=5,
        num_return_sequences=1)

jais_llm = HuggingFacePipeline(pipeline=pipeline)

In [53]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Use the given context to answer the question in Arabic language. REPLY ONLY IN ARABIC "
    "Use three sentence maximum and keep the answer precise and concise."
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [54]:
question_answer_chain = create_stuff_documents_chain(jais_llm , prompt)
chain = create_retrieval_chain(retriever_E5, question_answer_chain)

In [56]:
question = df['Questions'][2]

In [57]:
def extract_arabic_jais(text):
    # Check for "الجواب" scenario
    keyword = " :الجواب"
    keyword_index = text.find(keyword)
    if keyword_index != -1:
        start_index = keyword_index + len(keyword)
        return text[start_index:].strip()

    # Check for "AI:" scenario
    start_keyword = "AI:"
    start_index = text.find(start_keyword)
    if start_index != -1:
        start_index += len(start_keyword)
        remaining_text = text[start_index:]
        # Regex to find the start of English text
        end_match = re.search(r'[a-zA-Z]', remaining_text)
        if end_match:
            end_index = end_match.start()
            return remaining_text[:end_index].strip()
        return remaining_text.strip()

In [58]:
extract_arabic_jais(chain.invoke({"input": df['Questions'][2]})['answer'])

### Generation Answers

In [20]:
questions = df['Questions']
true_answers = df['Answers']

In [24]:
def generate_answer(question):
    answer = chain.invoke({"input": question})['answer']
    arabic_answer = extract_arabic_jais(answer)
    # while arabic_answer == None:
    #     answer = chain.invoke({"input": question})['answer']
    #     arabic_answer = extract_arabic_jais(answer)
    return arabic_answer

In [25]:
from datasets import Dataset
# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(df)

# Apply the chain in parallel using Dataset's map function
results = dataset.map(lambda x: {"answer": generate_answer(x["Questions"])})


Map:   0%|          | 0/158 [00:00<?, ? examples/s]

In [26]:
results

Dataset({
    features: ['Retrieved_true', 'Questions', 'paraphrases', 'eg_ar_question (GPT3.5 Turbo)', 'eg_ar_question_edited ', 'answer (GPT3.5 Turbo)', 'Unnamed: 6', 'Unnamed: 7', 'Answers', 'Generated_Answers_Mistral_7b', 'Eval_Answers_Mistral_7b', 'generated_tokens', 'true_tokens', 'bleu_score_mistral', 'bleu_score_mistral.1', 'Generated_Answers_llama_70b', 'answer'],
    num_rows: 158
})

In [27]:
x=results.to_pandas()

In [29]:
x.to_csv('/content/Retrieval_Data_mod_v4.csv',index=False)

In [59]:
print(chain.invoke({"input": df['Questions'][2]})['answer'])

System: Use the given context to answer the question in Arabic language. REPLY ONLY IN ARABIC Use three sentence maximum and keep the answer precise and concise.Context: الشارع فى المقهى، حيث تصبح جزءا من بضائع الدكاكين وحركة المرور، ومن الصمت أن تختار لملابسك

وأثاث منزلك ألوانا خافتة هادئة يرتاح إليها البصر، كما أن من الجلبة والصياح أن تختار هذه الأشياء من

ذوات الألوان الزاعقة التى تلفت الأنظار، ومن الصمت أن تعلن عن عيادتك، أو مكتبك، أو دكانك بلافتة

صغيرة متواضعة، كما أن من الجلبة والصياح أن تعلن عن نفسك بلافتة عريضة تسد على الناس مسالك

الطريق، واذ كر أن ارتفاع الصوت قد يدل على تفاهة الصائت فالكلب الذى ينبح لا يعض و كلما ازدادت

الشاة صياحا قل على ظهرها الصوف، والضفدعة الهزيلة الضئيلة تملأ الآفاق ضجة ونقيقا.

يستحيل أن تكون من الصاخبين ومن العاملين فى وقت واحد، ويستحيل أن تكون من الصائحين ومن

الفكرين فى وقت واحد، فقد يتعذر أن يجتمع الكلام والعمل لأن الفكرة إذا طافت برأسك فصحت بها

كلاما انتهى بذلك أمرها؛ أما إذا حبستها فى نفسك، وأغلقت دونها صدرك بمغاليق الصمت، فقد تتفجر

فى صورة 

In [ ]:
x['answer'].isnull().sum()

In [ ]:
# Generate answers using the chain
generated_answers = []
for q in questions:
    ans = extract_arabic_jais(chain.invoke({"input": q})['answer'])
    while ans is None:
        ans = extract_arabic_jais(chain.invoke({"input": q})['answer'])
    generated_answers.append(ans)

In [ ]:
len(generated_answers)

In [ ]:
df['Generated_Answers_jais_7b_quantized'] = generated_answers
df.to_csv('/content/Retrieval_Data_mod_v4.csv', index=False)